# Apredizagem por Regras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as srn
import plotly
import pickle
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import classification_report

In [9]:
#!pip install Orange3
import Orange

# Base Risco de Crédito

## Pré-processamento

In [13]:
df_risco = Orange.data.Table('../Machine Learning e Data Science com Python de A à Z/Bases de dados/risco_credito_regras.csv')
df_risco

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

## OneR

Pode-se usar o Weka explorer

## Prism

# Base Credit Data

## Pré-processamento

In [10]:
df_credit = pd.read_csv('../Machine Learning e Data Science com Python de A à Z/Bases de dados/credit_data.csv')
print(df_credit.shape)
df_credit.head()

(2000, 5)


,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


# Base Census

## Pré-processamento

## OneR

Pode-se usar o Weka explorer

## Prism